# Numeric Representations of Words

# Load the model that we created in Part 2

In [1]:
#!pip install import_ipynb

  Running setup.py bdist_wheel for import-ipynb ... done
  Stored in directory: /Users/song-yeongsug/Library/Caches/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
tensorflow-tensorboard 1.5.1 has requirement html5lib==0.9999999, but you'll have html5lib 1.0b8 which is incompatible.
bleach 1.5.0 has requirement html5lib!=0.9999,!=0.99999,<0.99999999,>=0.999, but you'll have html5lib 1.0b8 which is incompatible.


In [11]:
#ipynb import another ipynb file

import import_ipynb
import introduction

In [12]:
from gensim.models import Word2Vec

In [13]:
model = Word2Vec.load("300features_40minwords_10context")

In [14]:
type(model.wv.syn0) 

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


numpy.ndarray

In [15]:
model.wv.syn0.shape

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


(16490, 300)

In [16]:
model["flower"]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 7.41537986e-03,  5.23609342e-03, -4.97090630e-02,  8.38028863e-02,
        5.72579652e-02, -1.81115605e-02, -2.40733977e-02,  5.31086102e-02,
       -9.05622393e-02, -2.49681659e-02,  8.96626860e-02,  6.40268549e-02,
        3.85011220e-03, -3.27252261e-02, -2.00048592e-02,  6.80855438e-02,
        5.85490316e-02, -1.04972877e-01,  4.75187115e-02, -5.35832234e-02,
        1.48192778e-01,  2.95738764e-02, -7.10599218e-03, -5.96239567e-02,
       -2.55525373e-02, -3.20339054e-02, -2.33951360e-02, -3.61214019e-02,
        3.54553871e-02, -6.82883197e-03,  6.23100512e-02, -9.27016605e-03,
        1.40505955e-02,  6.71304241e-02, -8.52814242e-02, -7.28879729e-03,
        7.44247139e-02, -1.94614276e-03, -2.97842007e-02,  9.05904733e-03,
       -1.50373548e-01,  1.34634625e-04,  3.28904577e-02, -1.41076068e-03,
        3.45085189e-02,  4.00814984e-04, -5.72280167e-03,  2.55060606e-02,
       -1.51019856e-01, -1.03552513e-01,  4.38100612e-03,  7.23964497e-02,
       -8.78470391e-02, -

# From Words To Paragraphs, Attempt 1: Vector Averaging

In [17]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print("Review %d of %d" % (counter, len(reviews)))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs

In [10]:
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

NameError: name 'review_to_wordlist' is not defined

In [ ]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print "Fitting a random forest to labeled training data..."
forest = forest.fit( trainDataVecs, train["sentiment"] )

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )
